In [1]:
from Functions.fn_basic import readFiles, arrayToDataframe, channelsToRegions

import os
import numpy as np
import mne
import pandas as pd
import mne_features

In [2]:
exp_folder = input('Experiment folder (e.g., Eyes Open\Baseline):')
#exp_folder = r"Eyes Closed\test"
exp_condition = input('Experiment condition in short (e.g., EO_07)')

clean_folder = r"Data\Clean"
dir_inprogress = os.path.join(clean_folder,exp_folder)

file_dirs, subject_names = readFiles(dir_inprogress,"_clean-epo.fif")

Files in folder: 2


In [3]:
df_hfd = pd.DataFrame()
# Go through all the files (subjects) in the folder
for i in range(len(file_dirs)):
    # Read the clean data from the disk
    epochs = mne.read_epochs(fname='{}\{}_clean-epo.fif'.format(dir_inprogress,subject_names[i]))
    epochs_array = epochs.get_data()
    n_channels = len(epochs.info.ch_names)

    epochs_contin = np.empty(shape=(n_channels,1))
    for j in range(len(epochs_array[:,0,0])):
        epo_iter = epochs_array[j,:,:]
        epochs_contin = np.append(epochs_contin,epo_iter,axis=1)
    epochs_contin = epochs_contin[:,1:]

    hfd = mne_features.univariate.compute_higuchi_fd(data=epochs_contin,kmax=120)
    df_hfd_temp = arrayToDataframe(subject_names[i],epochs,hfd)
    df_hfd = pd.concat([df_hfd,df_hfd_temp])
    
df_hfd_regional = channelsToRegions(df_hfd)

try:
    os.makedirs(os.path.join(r"Results\Higuchi",exp_folder))
except FileExistsError:
    pass

df_hfd_regional.to_excel(r"Results\Higuchi\{}\{}_hfd_regional.xlsx".format(exp_folder,exp_condition))
df_hfd.to_excel(r"Results\Higuchi\{}\{}_hfd_channels.xlsx".format(exp_folder,exp_condition))


Reading c:\Users\TANIJARV\Downloads\Ketamine-SI project\Data\Clean\Eyes Closed\test\OKTOS_0001_00A_EC_clean-epo.fif ...
    Read a total of 1 projection items:
        EOG-eeg--0.200-0.200-PCA-01 (1 x 32) active
    Found the data of interest:
        t =       0.00 ...    4999.02 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
52 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Reading c:\Users\TANIJARV\Downloads\Ketamine-SI project\Data\Clean\Eyes Closed\test\OKTOS_0002_00A_EC_clean-epo.fif ...
    Read a total of 1 projection items:
        EOG-eeg--0.200-0.200-PCA-01 (1 x 32) active
    Found the data of interest:
        t =       0.00 ...    4999.02 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
47 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection it